In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leaf-classification/train.csv.zip
/kaggle/input/leaf-classification/test.csv.zip
/kaggle/input/leaf-classification/sample_submission.csv.zip
/kaggle/input/leaf-classification/images.zip


**Importing Libraries**

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, GridSearchCV

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

**Loading the data**

In [3]:
train = pd.read_csv("/kaggle/input/leaf-classification/train.csv.zip")
test = pd.read_csv("/kaggle/input/leaf-classification/test.csv.zip")

**Exploration of data**

In [4]:
train.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Columns: 194 entries, id to texture64
dtypes: float64(192), int64(1), object(1)
memory usage: 1.5+ MB


**990 samples, 192 features**

In [6]:
train['species'].nunique()

99

**99 unique species**

In [7]:
train['species'].value_counts()

Alnus_Cordata                10
Quercus_Canariensis          10
Populus_Nigra                10
Betula_Austrosinensis        10
Ilex_Cornuta                 10
                             ..
Quercus_Imbricaria           10
Lithocarpus_Cleistocarpus    10
Quercus_Agrifolia            10
Pterocarya_Stenoptera        10
Quercus_Rhysophylla          10
Name: species, Length: 99, dtype: int64

**each species has 10 samples in training set**

In [8]:
test.head()

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,4,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,...,0.006836,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711
1,7,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,...,0.000000,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922
2,9,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,...,0.128910,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000
3,12,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,...,0.012695,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789
4,13,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,...,0.000000,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Columns: 193 entries, id to texture64
dtypes: float64(192), int64(1)
memory usage: 895.8 KB


**Target: classify 594 test samples into 99 species**

**Preprocess data**

**1. Label encoder**

In [10]:
le = LabelEncoder().fit(train['species'])

**encode species in training set**

In [11]:
train['label'] = le.transform(train['species'])

**drop id & species columns. seperate labels in training set**

In [12]:
labels = train['label']
train_df = train.drop(columns=['id','species','label'])

**2. Feature scaling**

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(train_df)
train_scale = pd.DataFrame(scaler.transform(train_df))

**3. Split train - validation set**

In [14]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for train_index, val_index in sss.split(train_scale,  labels):
    x_train, x_val = train_scale.iloc[train_index], train_scale.iloc[val_index]
    y_train, y_val = labels.iloc[train_index], labels.iloc[val_index]

In [15]:
print(x_train.shape,y_train.shape)
print(x_val.shape, y_val.shape)

(792, 192) (792,)
(198, 192) (198,)


In [16]:
y_train.value_counts()

98    8
24    8
26    8
27    8
28    8
     ..
68    8
69    8
70    8
71    8
0     8
Name: label, Length: 99, dtype: int64

**each class has 8 samples in training set**

**Save test set id & features with scaling**

In [17]:
test_id = test['id']
test_features = test.drop('id', axis=1)
test_features_scale = scaler.transform(test_features)

**Model selection, model training & fine tuning**

**Three classifiers are considered in this notebook: Random forest classifier, Support vector machine classifier and KNeighbors**
**classifier.**

**GridSearchCV is used to fine-tune some hyperparameters**

In [18]:
cv_sets = ShuffleSplit(n_splits=10,test_size=0.20,random_state=42)
classifiers = [RandomForestClassifier(), SVC(), KNeighborsClassifier()]
params = [{'n_estimators' : [3,10,30], 'max_features':[2,4,6,8]},
          {'kernel':('linear','poly','sigmoid','rbf'),'C':[0.01,0.05,0.025,0.07,0.09,1.0], 'gamma':['scale'], 'probability':[True]},
          {'n_neighbors': [3,5,7,9]}]

In [19]:
best_estimators = []
for classifier, param in zip(classifiers, params):
    grid = GridSearchCV(classifier,param,cv=cv_sets)
    grid = grid.fit(x_train,y_train)
    best_estimators.append(grid.best_estimator_)

In [20]:
best_estimators

[RandomForestClassifier(max_features=8, n_estimators=30),
 SVC(C=0.025, kernel='linear', probability=True),
 KNeighborsClassifier(n_neighbors=3)]

**Deploy models on validation set & choose the best one**

In [21]:
for estimator in best_estimators:
    estimator.fit(x_train, y_train)
    name = estimator.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    print('**Training set**')
    train_predictions = estimator.predict(x_train)
    acc = accuracy_score(y_train, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    train_predictions = estimator.predict_proba(x_train)
    ll = log_loss(y_train, train_predictions)
    print("Log Loss: {}".format(ll))
    
    print('**Validation set**')
    train_predictions = estimator.predict(x_val)
    acc = accuracy_score(y_val, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    train_predictions = estimator.predict_proba(x_val)
    ll = log_loss(y_val, train_predictions)
    print("Log Loss: {}".format(ll))
    
print("="*30)

RandomForestClassifier
****Results****
**Training set**
Accuracy: 100.0000%
Log Loss: 0.2379167513856075
**Validation set**
Accuracy: 96.9697%
Log Loss: 0.7918659059436718
SVC
****Results****
**Training set**
Accuracy: 100.0000%
Log Loss: 2.1809267600143
**Validation set**
Accuracy: 99.4949%
Log Loss: 2.2941920307396058
KNeighborsClassifier
****Results****
**Training set**
Accuracy: 98.2323%
Log Loss: 0.041433802638993475
**Validation set**
Accuracy: 98.4848%
Log Loss: 0.0412192836531407


**Deploy the chosen model on test set**

In [22]:
pred = best_estimators[2].predict_proba(test_features_scale)

In [23]:
pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])